# 十種猴子影像辨識

## 導入模組

In [86]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, Conv, MaxPool, onecold, onehotbatch, throttle, logitcrossentropy
using Augmentor
using Statistics
using Images
using JLD

## 載入資料

In [87]:
pl = Resize(224, 224) |>
     Either(1=>FlipX(), 1=>NoOp()) |>
     Either(1=>Zoom(0.9:0.1:1.2), 1=>NoOp()) |>
     CropSize(224, 224)

4-step Augmentor.ImmutablePipeline:
 1.) Resize to 224×224
 2.) Either: (50%) Flip the X axis. (50%) No operation.
 3.) Either: (50%) Zoom by I ∈ {0.9×0.9, 1.0×1.0, 1.1×1.1, 1.2×1.2}. (50%) No operation.
 4.) Crop a 224×224 window around the center

### 載入訓練資料

In [88]:
monkeys = Array{Float64}(undef, 1097, 224, 224, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(224, 224, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:224)
                for k in (1:224)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

### 儲存訓練資料

In [89]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [90]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 224, 224, 3, :)
train_x = Float32.(train_x)

224×224×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.517647    0.239216   1.0        …  0.576471   0.94902   0.462745
 0.301961    0.109804   0.364706      0.435294   0.32549   0.486275
 0.235294    0.160784   0.619608      0.0705882  0.870588  0.192157
 1.0         0.772549   0.431373      0.0784314  0.619608  0.443137
 0.223529    0.933333   0.27451       0.419608   0.517647  0.592157
 0.411765    0.333333   0.0196078  …  0.65098    0.596078  0.392157
 0.494118    0.113725   0.509804      0.176471   0.160784  0.329412
 0.25098     0.278431   0.0470588     0.466667   0.337255  0.2
 0.576471    0.168627   0.14902       0.447059   0.639216  0.345098
 0.254902    0.45098    0.0431373     0.447059   0.780392  0.776471
 0.247059    0.639216   0.0980392  …  0.372549   0.572549  0.298039
 0.423529    0.509804   0.627451      0.109804   0.258824  0.729412
 0.172549    0.121569   0.686275      0.505882   0.847059  0.184314
 ⋮                                 ⋱                       
 0.494118    

### 載入驗證資料

In [91]:
val_monkeys = Array{Float64}(undef, 272, 224, 224, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(224, 224, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:224)
                for k in (1:224)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [92]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [93]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 224, 224, 3, :)
test_x = Float32.(test_x)

224×224×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.270588   0.223529    0.141176   …  0.894118   0.890196    0.423529
 0.517647   0.258824    0.160784      0.470588   0.207843    0.12549
 0.886275   0.00392157  0.356863      0.839216   0.301961    0.615686
 0.266667   0.266667    0.592157      0.780392   0.270588    0.0666667
 0.623529   0.501961    0.352941      0.654902   0.0470588   0.239216
 0.145098   0.294118    0.745098   …  0.737255   0.486275    0.176471
 0.913725   0.235294    0.152941      0.984314   0.00392157  0.294118
 0.8        0.12549     0.458824      0.247059   0.466667    0.419608
 0.6        0.396078    0.556863      0.666667   0.203922    0.545098
 0.564706   0.0705882   0.545098      0.72549    0.0117647   0.109804
 0.74902    0.458824    0.14902    …  0.447059   0.54902     0.568627
 0.788235   0.156863    0.443137      0.458824   0.0313726   0.0980392
 0.301961   0.27451     0.105882      0.819608   0.329412    0.294118
 ⋮                                 ⋱      

### 訓練標籤

In [94]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [95]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [96]:
batchsize = 32
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.27058825 0.22352941 … 0.8901961 0.42352942; 0.5176471 0.25882354 … 0.20784314 0.1254902; … ; 0.08235294 0.7372549 … 0.44705883 0.16078432; 1.0 0.54509807 … 0.72156864 0.49019608]

Float32[0.25882354 0.24705882 … 0.28235295 0.2509804; 0.8901961 0.35686275 … 0.5019608 0.5254902; … ; 0.31764707 0.24313726 … 0.21568628 0.0; 0.5921569 0.6039216 … 0.5294118 0.52156866]

Float32[0.14509805 0.13333334 … 0.24705882 0.24313726; 0.5568628 0.3019608 … 1.0 0.21960784; … ; 0.3254902 0.43137255 … 0.5019608 0.22745098; 0.69411767 0.1254902 … 0.15686275 0.35686275]

Float32[0.13725491 0.53333336 … 0.4745098 0.49411765; 0.46666667 0.22745098 … 0.12156863 0.34901962; … ; 0.101960786 0.5803922 … 0.7294118 0.019607844; 0.3254902 0.27058825 … 0.3529412 0.011764706]

Float32[0.13333334 0.21568628 … 0.38431373 0.08627451; 0.10980392 0.45490196 … 0.40392157 0.36078432; … ; 0.52156866 0.15686275 … 0.23137255 0.9764706; 0.41568628 1.0 … 0.78039217 0.79607844]

Float32[0.011764706 0.21960784

## CNN 模型

In [112]:
model = Chain(
    
    # First Block
    Conv((3, 3), pad = (1, 1), 3 => 64, relu), # 224x224
    Dropout(0.3f0),
    Conv((3, 3), pad = (1, 1), 64 => 64, relu), # 224x224
    MaxPool((2, 2), stride = (2, 2)), # 112x112
    
    # Second Block
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 112x112
    Dropout(0.4f0),
    Conv((3, 3), 128 => 128, pad = (1, 1), relu), # 112x112
    MaxPool((2, 2), stride = (2, 2)), # 56x56
    
    # Third Block
    Conv((3, 3), 128 => 256, pad = (1, 1), relu), # 56x56
    Dropout(0.4f0),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu), # 56x56
    Dropout(0.4f0),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu), # 56x56
    MaxPool((2, 2), stride = (2, 2)), # 28x28
    
    # Forth Block
    Conv((3, 3), 256 => 512, pad = (1, 1), relu), # 28x28
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 28x28
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 28x28
    MaxPool((2, 2), stride = (2, 2)), # 14x14
    
    # Fifth Block
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 14x14
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 14x14
    Dropout(0.4f0),
    Conv((3, 3), 512 => 512, pad = (1, 1), relu), # 14x14
    MaxPool((2, 2), stride = (2, 2)), # 7x7
    
    # Flatten & Dense
    flatten,
    Dense(25088, 1024),
    Dropout(0.5f0),
    Dense(1024, 256),
    Dropout(0.5f0),
    Dense(256, 10))

Chain(Conv((3, 3), 3=>64, relu), Dropout(0.3), Conv((3, 3), 64=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>128, relu), Dropout(0.4), Conv((3, 3), 128=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 256=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), Dropout(0.4), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(25088, 1024), Dropout(0.5), Dense(1024, 256), Dropout(0.5), Dense(256, 10))

In [113]:
size(train_x)

(224, 224, 3, 1097)

In [114]:
model(train_x[:, :, :, 1:3])

10×3 Array{Float32,2}:
  0.000810899   0.000654559   0.00118619
 -0.00289766   -0.00326309   -0.00329005
  0.000605787   0.000907054   0.000566911
 -0.00367184   -0.00387983   -0.00328652
 -0.00350932   -0.0035228    -0.00313218
  0.0023994     0.0024192     0.00234842
  0.00730776    0.00725329    0.00706215
 -0.00314078   -0.00326512   -0.00336667
 -0.00277518   -0.00253365   -0.00292452
 -0.00527976   -0.00486611   -0.00462681

## 損失函數

In [115]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

In [116]:
accuracy(x, y, model) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))

accuracy (generic function with 1 method)

## Callback 函式

In [118]:
function callback()
    print("Testing ")
    @show loss(test_x, test_y)
    print("Testing ")
    @show accuracy(test_x, test_y, model)
    println()
    accuracy(test_x, test_y, model) > 0.8 && Flux.stop()
end

callback (generic function with 1 method)

## 模型訓練

In [119]:
epochs = 30
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.001), cb = throttle(callback, 250))

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing 

OutOfMemoryError: OutOfMemoryError()

## 模型評估

In [16]:
accuracy(test_x, test_y)

0.09926470588235294